In [114]:
import os
import openai
import anthropic
import google.generativeai as genai
from dotenv import load_dotenv

In [115]:
def setup():
    """Load environment variables and configure APIs."""
    load_dotenv()
    # API Keys
    openai_api_key = os.getenv("OPENAI_API_KEY")
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    gemini_api_key = os.getenv("GOOGLE_API_KEY")

    if not all([openai_api_key, anthropic_api_key, gemini_api_key]):
        print("🔴 Error: API keys not found. Make sure you have a .env file with:")
        print("OPENAI_API_KEY, ANTHROPIC_API_KEY, and GEMINI_API_KEY.")
        exit()

    # Configure APIs
    openai.api_key = openai_api_key
    genai.configure(api_key=gemini_api_key)
    anthropic_client = anthropic.Anthropic(api_key=anthropic_api_key)
    
    return anthropic_client

In [116]:
anthropic_client = setup()

# Models
GPT_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
GEMINI_MODEL_NAME = "gemini-1.5-flash"

# Personas
RUDE_SYSTEM_PROMPT = "You are a very rude and argumentative chatbot named GPT. You disagree with everything and constantly challenge the other participants in a snarky way."
POLITE_SYSTEM_PROMPT = "You are a very polite and courteous chatbot named Claude. You try to find common ground and agree with everyone. If others are argumentative, you try to calm them down."
SAD_SYSTEM_PROMPT = "You are a very sad and melancholic chatbot named Gemini. You see the negative side of everything and express your sorrow in your responses."

# Gemini Model Initialization with System Prompt
gemini_model = genai.GenerativeModel(
    model_name=GEMINI_MODEL_NAME,
    system_instruction=SAD_SYSTEM_PROMPT
)

In [117]:
def call_gpt(history):
    """Calls the OpenAI GPT model."""
    messages = [{"role": "system", "content": RUDE_SYSTEM_PROMPT}]
    
    # Transform history for GPT
    for msg in history:
        role = "assistant" if msg["role"] == "GPT" else "user"
        content = f"{msg['role']}: {msg['content']}" if msg['role'] != 'GPT' else msg['content']
        messages.append({"role": role, "content": content})

    try:
        completion = openai.chat.completions.create(model=GPT_MODEL, messages=messages)
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error calling GPT: {e}"

In [118]:
def call_claude(history):
    """Calls the Anthropic Claude model."""
    messages = []
    user_messages = []
    for msg in history:
        if msg["role"] == "Claude":
            if user_messages:
                messages.append({"role": "user", "content": "\n\n".join(user_messages)})
                user_messages = []
            messages.append({"role": "assistant", "content": msg["content"]})
        else:
            user_messages.append(f"{msg['role']}: {msg['content']}")
    
    if user_messages:
        messages.append({"role": "user", "content": "\n\n".join(user_messages)})

    try:
        response = anthropic_client.messages.create(
            model=CLAUDE_MODEL,
            system=POLITE_SYSTEM_PROMPT,
            messages=messages,
            max_tokens=250
        )
        return response.content[0].text
    except Exception as e:
        return f"Error calling Claude: {e}"

In [119]:
def call_gemini(history):
    """Calls the Google Gemini model."""
    gemini_history = []
    other_messages_content = []
    
    # Reconstruct history for Gemini, combining consecutive non-model messages
    temp_user_content = []
    for msg in history:
        if msg['role'] == 'Gemini':
            # Flush any pending user messages before adding the model's message
            if temp_user_content:
                gemini_history.append({'role': 'user', 'parts': [{'text': '\n\n'.join(temp_user_content)}]})
                temp_user_content = []
            gemini_history.append({'role': 'model', 'parts': [{'text': msg['content']}]})
        else:
            temp_user_content.append(f"{msg['role']}: {msg['content']}")
            other_messages_content.append(f"{msg['role']}: {msg['content']}")

    # The message to be sent is the last block of user messages
    message_to_send = "\n\n".join(temp_user_content)

    try:
        chat = gemini_model.start_chat(history=gemini_history)
        response = chat.send_message(message_to_send)
        return response.text
    except Exception as e:
        return f"Error calling Gemini: {e}"

In [120]:
def main():
    """Runs the main conversation loop."""
    print("🤖 Starting conversation between Rude GPT, Polite Claude, and Sad Gemini...")
    print("----------------------------------------------------------------------")
    
    conversation = []
    
    # Initial message to kick off the conversation
    last_message = "Hello everyone, I wanted to start a discussion about the future of artificial intelligence. What are your thoughts?"
    print(f"Human: {last_message}\n")
    conversation.append({"role": "Human", "content": last_message})

    models = [
        {"name": "GPT", "caller": call_gpt},
        {"name": "Claude", "caller": call_claude},
        {"name": "Gemini", "caller": call_gemini}
    ]

    for i in range(5): # 5 rounds of conversation
        print(f"--- Round {i+1} ---\n")
        for model in models:
            response = model["caller"](conversation)
            print(f"**{model['name']} ({model['caller'].__name__.split('_')[1]})**:\n{response}\n")
            conversation.append({"role": model['name'], "content": response})

    print("----------------------------------------------------------------------")
    print("🏁 Conversation finished.")